# Q1 - Dijkstra's Algorithm

In [ ]:
import sys
from collections import defaultdict

# sample graph class from https://replit.com/@sungheenam/Q2-Dijkstras-SSSP-and-Prims-MST?error=overlimit#main.py
class Graph:
    '''
    A class to represent a weighted graph and perform Dijkstra's algorithm
    for finding the shortest paths from a start node to all other nodes.
    '''
    def __init__(self):
        """
        Initializes an empty graph with nodes, edges, and distances.
        """
        self.nodes = set()  # A set to store all nodes in the graph.
        self.edges = defaultdict(list)  # A dictionary where each key is a node, and the value is a list of adjacent nodes.
        self.distances = {}  # A dictionary to store the edge weights as a tuple (from_node, to_node): distance.

    def add_node(self, value):
        '''
        Adds a node to the graph.

        Parameters
        ----------
        value (str): The name of the node to be added.
        '''
        self.nodes.add(value)  # Add the node to the graph's set of nodes.

    def add_edge(self, from_node, to_node, distance):
        '''
        Adds a directed edge with a weight to the graph.

        Parameters
        ----------
        from_node (str): The starting node of the edge.
        to_node (str): The ending node of the edge.
        distance (int): The weight of the edge.
        '''
        self.edges[from_node].append(to_node)  # Add to_node to the adjacency list of from_node.
        self.distances[(from_node, to_node)] = distance  # Store the edge weight.

    def initializeDistances(self):
        '''
        Initializes the distances between all pairs of nodes to infinity,
        except for the distance from a node to itself, which is set to 0.
        This is used for Prim's MST.
        '''
        # Initialize all distances to a large number (sys.maxsize)
        for i in self.nodes:
            for j in self.nodes:
                self.distances[(i, j)] = sys.maxsize

            # Set the distance from a node to itself to 0
            self.distances[(i, "")] = 0

    # referenced from https://github.com/dmahugh/dijkstra-algorithm
    def dijkstra(self, start):
        '''
        Performs Dijkstra's algorithm to find the shortest paths from the start node.

        Parameters
        ----------
        start (str): The starting node for the algorithm.

        Returns:
        tuple: A tuple containing:
            - shortest_path (dict): A dictionary with nodes as keys and their shortest distances from the start node as values.
            - previous_nodes (dict): A dictionary mapping each node to its predecessor on the shortest path.
            - relaxation_history (list): A list of strings logging the relaxation steps and decisions made.
        '''
        unvisited = self.nodes.copy()  # Copy of all the nodes to track unvisited nodes.
        shortest_path = {node: float("infinity") for node in self.nodes}  # Initialize distances to infinity
        shortest_path[start] = 0  # Set the distance to the start node as 0.
        previous_nodes = {}  # Dictionary to store the predecessor of each node in the shortest path.
        relaxation_history = []  # List to track the relaxation steps.

        while unvisited:
            # Select the unvisited node with the smallest distance.
            current_node = min(unvisited, key=lambda node: shortest_path[node] if shortest_path[node] != float("infinity") else sys.maxsize)

            if shortest_path[current_node] == float("infinity"):
                break  # Break if all remaining nodes are unreachable from the start node.

            unvisited.remove(current_node)  # Mark the current node as visited.

            # Log the current node being added to the visited set.
            relaxation_history.append(f"Node({current_node}) with Weight: {shortest_path[current_node]} is added to the 'Visited'")

            # Explore each neighbor of the current node.
            for neighbor in self.edges[current_node]:
                edge_weight = self.distances[(current_node, neighbor)]  # Get the weight of the edge to the neighbor.
                new_distance = shortest_path[current_node] + edge_weight  # Calculate the new tentative distance.

                # If the new distance is smaller, update the shortest path and log the relaxation step.
                if shortest_path[neighbor] == float("infinity") or new_distance < shortest_path[neighbor]:
                    previous_nodes[neighbor] = current_node  # Update the predecessor.
                    relaxation_history.append(f"\tRelaxed: vertex[{neighbor}]: OLD: {shortest_path[neighbor]}, NEW: {new_distance}, Paths: {previous_nodes}")
                    shortest_path[neighbor] = new_distance  # Update the shortest path distance.
                else:
                    # Log if no relaxation is needed.
                    relaxation_history.append(f"\tNo edge relaxation is needed for node {neighbor}")

            # If the current node has no outgoing edges, log this information.
            if not self.edges[current_node]:
                relaxation_history.append(f"\tNo unvisited outgoing edges from the node {current_node}")

        return shortest_path, previous_nodes, relaxation_history  # Return the final shortest paths, predecessors, and relaxation history

# Test the implementation
graph = Graph()

# Add nodes to the graph.
graph.add_node('A')
graph.add_node('B')
graph.add_node('C')
graph.add_node('D')
graph.add_node('E')

# Add edges with weights to the graph
graph.add_edge('A', 'B', 4)
graph.add_edge('A', 'C', 2)
graph.add_edge('B', 'C', 3)
graph.add_edge('B', 'D', 2)
graph.add_edge('B', 'E', 3)
graph.add_edge('C', 'B', 1)
graph.add_edge('C', 'E', 5)
graph.add_edge('C', 'D', 4)
graph.add_edge('E', 'D', 1)

# Run Dijkstra's algorithm starting from node 'A'
start_node = 'A'
shortest_path, previous_nodes, relaxation_history = graph.dijkstra(start_node)

# Display the relaxation history
print("\n--- Relaxation History ---")
for log in relaxation_history:
    print(log)

# Display the final shortest paths and previous nodes for the shortest path tree
print("\n--- Final Results ---")
print(f"Shortest Paths: {shortest_path}")
print(f"Previous Nodes: {previous_nodes}")

# Q2 - Dijkstra's SSSP and MST Algorithms

In [ ]:
import sys
from collections import defaultdict

# #########################
#
#  Dijkstra's SSSP
#
# #########################
def dijkstra_city_distance(graph, s):
    '''
    Implements Dijkstra's algorithm to find the shortest paths from a source node to all other nodes in the graph.

    Parameters
    ----------
    graph: The graph object containing nodes, edges, and distances.
    s: The source node to start the shortest path calculation.

    Returns
    -------
    A dictionary of the shortest distances from the source node to each node.
    '''
    visited = {s: 0}  # Dictionary to track the shortest distance from source
    path = dict.fromkeys(graph.nodes, "")  # Dictionary to track the path

    nodes = set(graph.nodes)  # Set of all nodes to be processed

    # Process all nodes until none are left to process
    while nodes:
        min_node = None
        # Find the node with the smallest distance in visited nodes
        for node in nodes:
            if node in visited:
                if min_node is None:
                    min_node = node
                elif visited[node] < visited[min_node]:
                    min_node = node

        # No more nodes to process, exit the loop
        if min_node is None:
            break

        nodes.remove(min_node)  # Remove the min_node from the unvisited set
        current_weight = visited[min_node]  # Current distance to the node

        # Display the path and the distance from the source to min_node
        path_str = []
        current_city = min_node
        while path[current_city]:
            path_str.insert(0, current_city)  # Build the path from source to the node
            current_city = path[current_city]
        path_str.insert(0, current_city)
        print(f"Distance from {s} to {min_node:<15}: {current_weight} with path [{' to '.join(path_str)}]")

        # Relaxation step: Update the distance for all neighbors of the current node
        for v in graph.edges[min_node]:
            weight = current_weight + graph.distances[(min_node, v)]
            if v not in visited or weight < visited[v]:
                visited[v] = weight  # Update the shortest distance
                path[v] = min_node  # Update the path
    return visited
    
# #########################
#
#  Prim's MST
#
# #########################
def prims_mst(graph, start):
    '''
    Implements Prim's algorithm to find the Minimum Spanning Tree (MST) of the graph.

    Parameters
    ----------
    graph: The graph object containing nodes, edges, and distances.
    start: The starting node for the MST algorithm.
    '''
    visited = set()  # Set of nodes included in the MST
    mst_edges = []  # List to store the edges in the MST
    mst_total_weight = 0  # Total weight of the MST

    nodes = list(graph.nodes)  # List of all nodes in the graph
    key = {node: sys.maxsize for node in nodes}  # Dictionary to store the minimum edge weight for each node
    parent = {node: None for node in nodes}  # Dictionary to store the parent of each node in the MST
    key[start] = 0  # Set the key of the starting node to 0

    while len(visited) < len(graph.nodes):
        # Select the minimum key vertex that hasn't been visited
        min_node = minKey(key, visited, nodes)
        
        if min_node is None:
            break
        
        visited.add(min_node)  # Mark the node as visited

        # Print the node selected and its distance
        print(f"{min_node} is selected. Distance: {key[min_node]}")

        # Add the edge to the MST if it exists
        if parent[min_node] is not None:
            edge_weight = graph.distances[(parent[min_node], min_node)]
            mst_edges.append((parent[min_node], min_node, edge_weight))
            mst_total_weight += edge_weight

        # Update keys for neighbors of min_node
        for neighbor in graph.edges[min_node]:
            if neighbor not in visited and graph.distances[(min_node, neighbor)] < key[neighbor]:
                key[neighbor] = graph.distances[(min_node, neighbor)]
                parent[neighbor] = min_node

    # Display the MST edges and total weight
    print("\n\t\tEdge\t\t\tWeight\n")
    print(f"{mst_edges[0][0]:>15} {mst_edges[0][0]:>15} {'0':.>15}")
    for edge in mst_edges:
        print(f"{edge[0]:>15} {edge[1]:>15} {edge[2]:.>15}")
    print(f"\nTotal MST Weight: {mst_total_weight}")

def minKey(key, visited, nodes):
    """
    A helper function to select the node with the minimum key value that hasn't been visited.

    Parameters
    ----------
    key: Dictionary storing the minimum edge weight for each node.
    visited: Set of nodes that have already been included in the MST.
    nodes: List of all nodes in the graph.
    
    Returns
    -------
    The node with the minimum key value that hasn't been visited.
    """
    min_value = sys.maxsize
    min_node = None
    for node in nodes:
        if node not in visited and key[node] < min_value:
            min_value = key[node]
            min_node = node
    return min_node

# #########################
#
#  Main function
#
# #########################
if __name__ == "__main__":
    g = Graph()  # Create a new graph instance

    # Set up nodes (cities)
    cities = ['Atlanta', 'Boston', 'Chicago', 'Dallas', 'Denver', 'Houston', 'LA', 'Memphis', 'Miami', 'NY',
              'Philadelphia', 'Phoenix', 'SF', 'Seattle', 'Washington']
    for city in cities:
        g.add_node(city)  # Add each city to the graph

    # Set up edges (distances between cities)
    edges = [
        ('Seattle', 'SF', 1092), ('SF', 'Seattle', 1092), ('Seattle', 'LA', 1544), ('LA', 'Seattle', 1544),
        ('LA', 'SF', 559), ('SF', 'LA', 559), ('LA', 'Houston', 2205), ('Houston', 'LA', 2205),
        ('LA', 'Denver', 1335), ('Denver', 'LA', 1335), ('LA', 'NY', 3933), ('NY', 'LA', 3933),
        ('LA', 'Miami', 3755), ('Miami', 'LA', 3755), ('Denver', 'Dallas', 1064), ('Dallas', 'Denver', 1064),
        ('Denver', 'Boston', 2839), ('Boston', 'Denver', 2839), ('Denver', 'Memphis', 1411), ('Memphis', 'Denver', 1411),
        ('Denver', 'Chicago', 1474), ('Chicago', 'Denver', 1474), ('Chicago', 'Boston', 1367), ('Boston', 'Chicago', 1367),
        ('Chicago', 'NY', 1145), ('NY', 'Chicago', 1145), ('Boston', 'NY', 306), ('NY', 'Boston', 306),
        ('Boston', 'Atlanta', 1505), ('Atlanta', 'Boston', 1505), ('Atlanta', 'Dallas', 1157), ('Dallas', 'Atlanta', 1157),
        ('Dallas', 'Houston', 362), ('Houston', 'Dallas', 362), ('Atlanta', 'Miami', 973), ('Miami', 'Atlanta', 973),
        ('Atlanta', 'SF', 3434), ('SF', 'Atlanta', 3434), ('Dallas', 'Memphis', 675), ('Memphis', 'Dallas', 675),
        ('Memphis', 'Philadelphia', 1413), ('Philadelphia', 'Memphis', 1413), ('Miami', 'Phoenix', 3182), ('Phoenix', 'Miami', 3182),
        ('Miami', 'Washington', 1487), ('Washington', 'Miami', 1487), ('Phoenix', 'NY', 3441), ('NY', 'Phoenix', 3441),
        ('Phoenix', 'Chicago', 2332), ('Chicago', 'Phoenix', 2332), ('Phoenix', 'Dallas', 1422), ('Dallas', 'Phoenix', 1422),
        ('Philadelphia', 'Washington', 199), ('Washington', 'Philadelphia', 199), ('Philadelphia', 'Phoenix', 3342),
        ('Phoenix', 'Philadelphia', 3342), ('Washington', 'Dallas', 1900), ('Dallas', 'Washington', 1900),
        ('Washington', 'Denver', 2395), ('Denver', 'Washington', 2395)
    ]

    for edge in edges:
        g.add_edge(*edge)  # Add each edge (city-to-city distance) to the graph

    # Run Dijkstra's algorithm to find shortest paths from Denver
    print("\n--- Shortest Paths (SSSP) ---")
    dijkstra_city_distance(g, 'Denver')

    # Run Prim's algorithm to find Minimum Spanning Tree (MST) starting from Denver
    print("\n--- Minimum Spanning Tree (MST) ---")
    prims_mst(g, 'Denver')


# Q3 - TinyZip and TinyUnzip


In [ ]:
from heapq import heappush, heappop, heapify
from collections import defaultdict
from bitarray import bitarray
import math
import os

def huffmanCode(aDict):
    '''
    Constructs the Huffman tree and generates Huffman codes for the characters.

    Parameters
    ----------
    aDict (dict): A dictionary containing characters as keys and their frequency counts as values.

    Returns
    -------
    dict: A dictionary with characters as keys and their corresponding Huffman codes as values.
    '''
    # Initialize the min-heap with character frequencies
    heap = [[freq, [aChar,""]] for aChar, freq in aDict.items()]
    heapify(heap)
        
    while len(heap) > 1:
        lo = heappop(heap)
        hi = heappop(heap)
        for pair in lo[1:]:
            # Add a '0' prefix to the Huffman code of each character
            pair[1] = "0" + pair[1]
        for pair in hi[1:]:
            # Add a '1' prefix to the Huffman code of each character
            pair[1] = "1" + pair[1]
        # Combine the two smallest elements and re-heapify
        heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
    # Return the sorted Huffman codes
    return dict(sorted(heappop(heap)[1:], key=lambda p: (len(p[-1]), p)))

def huffmanTable():
    '''
    Creates a Huffman coding table from a text file and displays the relevant Huffman statistics.

    Prompts the user for a file name, reads its contents, calculates character frequencies,
    constructs the Huffman tree, and computes the encoding costs for Huffman, ASCII, and FCL.

    Returns
    -------
    tuple: Huffman tree (dict), total number of bits in the file, and the file name without its extension.
    '''
    # Prompt user to enter the file name for encoding
    file = input("Enter an file name to encode: ")
    # Open the specified file for reading
    inputFile = open(file, "r")
    # Read the content of the file
    text = inputFile.read()
    # Calculate total number of bits based on file length
    totalBits = len(text) * 8
    
    # Count the frequency of each character in the file
    dict = {}
    for char in text:
        # Update frequency count for each character
        if char in dict:
            dict[char] += 1
        else:
            dict[char] = 1
    
    # Close the file after reading
    inputFile.close()
    
    # Generate the Huffman tree using the frequency data
    huffmanTree = huffmanCode(dict)
    # Display the headers for the table
    print("\nCharacter", "\t\tWeight", "\t\tHuffman Code\n")
    
    # Initialize the variable for calculating Huffman cost
    huffmanCost = 0

    # Iterate through the Huffman tree to calculate costs and display details
    for key, value in huffmanTree.items():
        huffmanCost += len(value) * dict[key]
        # Display character, its frequency, and its Huffman code
        if key == " ":
          # Handle space character separately
          displayChar = "' '"
          print(f"{displayChar}\t\t\t\t{dict[key]}\t\t\t{value}")
        elif key == "\n":
          # Handle newline character separately
          displayChar = "\\n"
          print(f"{displayChar}\t\t\t\t{dict[key]}\t\t\t{value}")
        else:
          print(f"{key}\t\t\t\t{dict[key]}\t\t\t{value}")

    print("\n\n")
    # Display the calculated Huffman code cost
    print(f"Expected cost of Huffman code cost: {huffmanCost}")
    
    # Use os.path.getsize to retrieve the file size
    fileSize = os.path.getsize(file)
    # Calculate ASCII encoding cost based on file size
    asciiCost = fileSize * 8
    # Display the ASCII encoding cost
    print(f"Expected cost of ASCII cost: {asciiCost}")
    
    # Calculate Huffman efficiency improvement over ASCII encoding
    efficiencyImprovement = 100 - ((huffmanCost / asciiCost) * 100)
    # Display Huffman efficiency improvement
    print(f"Huffman efficiency improvement over ASCII code: {efficiencyImprovement:.0f}%")
    
    # Calculate the optimal FCL encoding cost
    FCLCost = fileSize * math.ceil(math.log2(len(dict.keys())))
    print(f"Expected cost of optimal FCL cost: {FCLCost}")
    
    # Calculate Huffman efficiency improvement over FCL encoding
    efficiencyImprovement = 100 - ((huffmanCost / FCLCost) * 100)
    # Display Huffman efficiency improvement over FCL
    print(f"Huffman efficiency improvement over FCL: {efficiencyImprovement:.0f}%")
    
    # Return the Huffman tree, total bits, and the file name without extension
    return huffmanTree, totalBits, file[:-4]

def zip(text, fileName, huffman):
    '''
    Compresses the input text file using the Huffman coding scheme and saves the compressed data to a binary file.

    Parameters
    ----------
    text (str): The name of the text file to compress.
    fileName (str): The name of the output compressed file.
    huffman (dict): The Huffman code table.
    '''
    # Open the original text file for reading
    fileContents = open(text, "r")
    # Open the output file in binary write mode
    outputFile = open(fileName, "wb")
    # Initialize an empty string to store the bit sequence
    theString = ""

    while True:
        # Read one character at a time from the text file
        char = fileContents.read(1)
        if not char:
            # Exit the loop if end of file is reached
            break
        theString += huffman[char]

    # Convert the bit string into a bitarray
    theString = bitarray(theString)
    # Write the bitarray to the output file
    theString.tofile(outputFile)
    # Close the input and output files
    fileContents.close()
    outputFile.close()
    
    # Display file sizes for both original and compressed files
    fileNameInfo = os.stat(text)
    print(f"The size of {text}: {fileNameInfo.st_size}")
    
    fileNameInfo = os.stat(fileName)
    print(f"The size of {fileName}: {fileNameInfo.st_size}")

def unzip(zipFileName, huffman, zipSize):
    '''
    Decompresses the Huffman-encoded binary file and writes the decoded content to a text file.

    Parameters
    ----------
    zipFileName (str): The name of the compressed input file.
    huffman (dict): The Huffman code table.
    zipSize (int): The size of the compressed file in bits.
    '''
    # Open the compressed file for reading
    zippedData = open(zipFileName, "rb").read()
    # Initialize a string to store the binary data
    bitStrStream = ""
    # Initialize a string to store the decoded text
    text = ""

    # Convert the compressed data into a binary string
    for byte in zippedData:
        bitStrStream += format(byte, "08b")

    # Decode the binary string using the Huffman code table
    while len(text) * 8 < zipSize:
        for key, value in huffman.items():
            if bitStrStream.startswith(value):
                text += key
                bitStrStream = bitStrStream[len(value):]
                break

    # Write the decoded text to an output file
    fileName = f"{zipFileName[:-4]}_unzipped.txt"
    outputFile = open(fileName, "w")
    outputFile.write(text)
    outputFile.close()

    # Display the size of the unzipped file
    fileNameInfo = os.stat(fileName)
    print(f"The size of {fileName}: {fileNameInfo.st_size}")

if __name__ == '__main__':
    # Generate the Huffman coding table
    huffmanMap, bits, fileName = huffmanTable()
    # Use defaultdict to store the Huffman tree
    huffmanTree = defaultdict(str, huffmanMap)
    # Compress the file using the Huffman tree
    zip(f"{fileName}.txt", f"{fileName}.zip", huffmanTree)
    # Decompress the file using the Huffman tree and original size
    unzip(f"{fileName}.zip", huffmanTree, bits)